In [1]:
# Импорт всех необходимых модулей для оптимизации
import sys
import os
from pathlib import Path

# Добавляем путь к модулям проекта
project_root = Path.cwd()
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Основные модули оптимизации
from optimization import (
    # Основные классы
    OptimizationExecutor,
    OptimizationReporter,
    
    # Хранилище результатов
    InMemoryResultsStore,
    OptimizationResult,
    ResultsStoreProtocol,
    
    # Пространство параметров
    ParameterDefinition,
    ParameterSpace,
    
    # Стратегии поиска
    SearchStrategy,
    GridSearchStrategy,
    RandomSearchStrategy,
)

# Дополнительные импорты для работы с данными и бэктестингом
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Optional, Mapping, Sequence
from dataclasses import dataclass
from datetime import datetime, UTC

# Импорты для работы с сигналами и индикаторами
from signals.base_signal import BaseSignal
from signals.signal_registry import SignalRegistry
from indicators.base_indicator import BaseIndicator
from indicators.indicator_registry import IndicatorRegistry

# Импорты для бэктестинга
from backtest.engine import BacktestEngine
from backtest.metrics import MetricsCalculator
from backtest.trade import Trade
from backtest.commission import CommissionCalculator
from optimization.executor import SignalConfig

print("✅ Все модули оптимизации успешно импортированы!")
print(f"📁 Рабочая директория: {project_root}")
print(f"📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy")
print(f"📊 Доступные хранилища: InMemoryResultsStore")
print(f"📈 Доступные репортеры: OptimizationReporter")


✅ Все модули оптимизации успешно импортированы!
📁 Рабочая директория: c:\Users\user\Documents\spreader_pro\code\trend_optimization
📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy
📊 Доступные хранилища: InMemoryResultsStore
📈 Доступные репортеры: OptimizationReporter


In [2]:
# Ячейка 1: Импорты и настройка
import sys
sys.path.append('src')
from data import CSVLoader
import pandas as pd
import numpy as np

from lightweight_charts import JupyterChart
# Ячейка 2: Настройка данных
DATA_DIR = r"G:\My Drive\data_fut"
TICKER = "GLDRUBF"
loader = CSVLoader(DATA_DIR, use_cache=True)

# Ячейка 3: Загрузка данных
start_date = '2025-01-15'
end_date = '2025-10-15'

df = loader.load(TICKER, timeframe=5, start_date=start_date, end_date=end_date)
print(f"Данные: {len(df)} баров, период: {df.index[0]} - {df.index[-1]}")

# Ячейка 4: Ваш анализ (например, Volatility Median)
# Здесь можете использовать функции из VM_base_optimize.py

Данные: 35279 баров, период: 2025-01-15 10:00:00 - 2025-10-14 23:50:00


In [4]:
# Простая функция вместо класса SignalFactory
def create_signal_config(parameters):
    """Разделяем параметры по назначению"""
    return SignalConfig(
        signal=SignalRegistry.get("SlopeSignal"),
        signal_params={'threshold': 0.0001},  # SlopeSignal: threshold, absolute
        indicator_params={
            'KATR': parameters.get('katr'),
            'PerATR': parameters.get('peratr'),
            'SMA': parameters.get('sma'),
            'MinRA': parameters.get('minra'),
            'FlATR': parameters.get('flatr'),
            'FlHL': parameters.get('flhl')
        },
        backtest_params={
            # entry_price_type/n_contracts можно переопределить здесь при необходимости
            # 'entry_price_type': 'close',
            # 'n_contracts': 1,
        }
    )

# Простая функция вместо класса IndicatorBuilder
def build_indicator(df, params):
    """Создаем индикатор с параметрами"""
    vm_ind = IndicatorRegistry.get("VolatilityMedian")
    return vm_ind.calculate(df, **params)

In [5]:
# Простой способ - создание из ваших данных
your_params = {
    "KATR": [2, 3, 4, 5, 6, 7, 8],
    "PerATR": [5, 20, 80], 
    "SMA": [1, 2, 3, 10],
    "MinRA": [0],
    "FlATR": [0],
    "FlHL": [0]
}

# Преобразование в ParameterSpace
definitions = []
for name, values in your_params.items():
    definitions.append(ParameterDefinition(
        name=name.lower(),  # приводим к нижнему регистру
        values=values
    ))

parameter_space = ParameterSpace.from_definitions(definitions)

In [13]:
# Создаем оптимизатор
executor = OptimizationExecutor(
    backtest_engine=BacktestEngine(),
    parameter_space=parameter_space,
    strategy=GridSearchStrategy(parameter_space),
    signal_factory=create_signal_config,  # ← Простая функция!
    results_store=InMemoryResultsStore(),
    indicator_builder=build_indicator,    # ← Простая функция!
    # ↓↓↓ новые настройки производительности/стабильности ↓↓↓
    checkpoint_interval=1000,              # было 1 → станет реже и быстрее
    per_candidate_timeout=None,           # убрать пер-кандидатные пулы
    default_max_workers=1,                # дефолт: одиночный режим
    checkpoint_enabled=True               # оставить включённым (но с редким интервалом)
    
)

print("✅ Оптимизатор создан!")

✅ Оптимизатор создан!


In [7]:
# === РАСЧЕТЫ ПЕРЕД ЗАПУСКОМ ===
print("📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ")
print("=" * 50)

# 1. Размер данных
print(f"📈 Будет протестировано: {len(df):,} свечей")
print(f"📅 Период данных: {df.index[0].strftime('%Y-%m-%d')} - {df.index[-1].strftime('%Y-%m-%d')}")

# 2. Расчет количества комбинаций
total_combinations = 1
param_details = []

for name, values in your_params.items():
    count = len(values)
    total_combinations *= count
    param_details.append(f"  {name}: {count} значений {values}")

print(f"\n🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:")
for detail in param_details:
    print(detail)

print(f"\n🧮 РАСЧЕТ КОМБИНАЦИЙ:")
print(f"  Всего параметров: {len(your_params)}")
print(f"  Комбинаций: {total_combinations:,}")
print(f"  Формула: {' × '.join([str(len(values)) for values in your_params.values()])} = {total_combinations:,}")

# 3. Оценка времени выполнения
estimated_time_per_combination = 0.1  # секунд на комбинацию (примерно)
total_time_seconds = total_combinations * estimated_time_per_combination
total_time_minutes = total_time_seconds / 60
total_time_hours = total_time_minutes / 60

print(f"\n⏱️ ОЦЕНКА ВРЕМЕНИ:")
print(f"  ~{estimated_time_per_combination} сек на комбинацию")
print(f"  Общее время: {total_time_seconds:.1f} сек ({total_time_minutes:.1f} мин)")
if total_time_hours >= 1:
    print(f"  Или: {total_time_hours:.1f} часов")

# 4. Предупреждения
if total_combinations > 10000:
    print(f"\n⚠️ ВНИМАНИЕ: Большое количество комбинаций!")
    print(f"   Рекомендуется уменьшить параметры или использовать RandomSearchStrategy")
    
if total_combinations > 100000:
    print(f"\n🚨 КРИТИЧНО: Очень много комбинаций!")
    print(f"   Рассмотрите возможность:")
    print(f"   - Уменьшения количества значений параметров")
    print(f"   - Использования RandomSearchStrategy с max_iterations=1000")
    print(f"   - Разделения оптимизации на этапы")

print("\n" + "=" * 50)



📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ
📈 Будет протестировано: 35,279 свечей
📅 Период данных: 2025-01-15 - 2025-10-14

🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:
  KATR: 7 значений [2, 3, 4, 5, 6, 7, 8]
  PerATR: 3 значений [5, 20, 80]
  SMA: 4 значений [1, 2, 3, 10]
  MinRA: 1 значений [0]
  FlATR: 1 значений [0]
  FlHL: 1 значений [0]

🧮 РАСЧЕТ КОМБИНАЦИЙ:
  Всего параметров: 6
  Комбинаций: 84
  Формула: 7 × 3 × 4 × 1 × 1 × 1 = 84

⏱️ ОЦЕНКА ВРЕМЕНИ:
  ~0.1 сек на комбинацию
  Общее время: 8.4 сек (0.1 мин)



In [11]:
import time
# === ЗАПУСКАЕМ ОПТИМИЗАЦИЮ ===
print("🚀 Начинаем оптимизацию...")
start_time = time.time()

run_dir = executor.run(df)  # фиксируем одиночный воркер

end_time = time.time()
actual_time = end_time - start_time

# === РЕЗУЛЬТАТЫ ===
results = list(executor.results_store)
print(f"✅ Завершено! Получено {len(results)} результатов")
print(f"⏱️ Фактическое время: {actual_time:.1f} сек ({actual_time/60:.1f} мин)")
print(f"📊 Скорость: {len(results)/actual_time:.1f} комбинаций/сек")
best_result = max(results, key=lambda r: r.metrics.get('total_return', 0))
print(f"🏆 Лучший результат: {best_result.parameters}")
print(f"📊 Метрики: {best_result.metrics}")
print(f"📁 Директория запуска: {run_dir}")

🚀 Начинаем оптимизацию...
✅ Завершено! Получено 84 результатов
⏱️ Фактическое время: 88.0 сек (1.5 мин)
📊 Скорость: 1.0 комбинаций/сек
🏆 Лучший результат: {'katr': 8, 'peratr': 5, 'sma': 10, 'minra': 0, 'flatr': 0, 'flhl': 0}
📊 Метрики: {'total_return': 0.014001053799999719, 'max_drawdown': 0.010070813944258239, 'recovery_factor': 1.3902603977687684, 'sharpe': 0.07823463472145951, 'sortino': 0.10378953259349757, 'n_flips': 90, 'n_trades': 91, 'win_rate': 0.42857142857142855, 'profit_factor': 1.3695894102704245}
📁 Директория запуска: output\optimization\20251022_141244


In [ ]:
# ## спящая ячейка для отображения графика
# df_chart = df.copy()
# df_chart = df_chart.rename(columns={
#     'OPEN': 'open',
#     'HIGH': 'high',
#     'LOW': 'low',
#     'CLOSE': 'close',
#     'VOL': 'volume'
# })

# print(f"Всего данных для отображения: {len(df_chart)} баров")

# # Вариант 1: Отобразить все данные (может быть медленно)
# chart = JupyterChart()
# chart.set(df_chart)
# chart.load()

In [ ]:

# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models import HoverTool



# output_notebook()
# p = figure(title='Equity (Bokeh)', x_axis_type='datetime', width=900, height=420, tools='pan,wheel_zoom,box_zoom,reset,save')
# p.line(equity.index, equity.values, line_width=2, color='#2962FF', legend_label='Equity')
# p.add_tools(HoverTool(tooltips=[('time', '@x{%F %T}'), ('equity', '@y')], formatters={'@x': 'datetime'}))
# p.legend.location = 'top_left'
# p.grid.grid_line_alpha = 0.25
# show(p)
